In [1]:
import os

import pandas as pd
from tqdm import tqdm

from utils.logparser import LogParser

In [2]:
df = pd.read_csv('./dataset/v1/train_clean.csv')
with open('./dataset/v1/log.log', "w") as f:
    for i in tqdm(range(len(df))):
        string = 'train ' + df['sn'].values[i]+' '+df['time'].values[i]+' '+df['server_model'].values[i]+' '+df['msg'].values[i]
        string = string.replace('  ', ' ')
        f.write(string)
        f.write('\n')

df = pd.read_csv('./dataset/v1/test_clean.csv')
with open('./dataset/v1/log.log', "a") as f:
    for i in tqdm(range(len(df))):
        string = 'test ' + df['sn'].values[i]+' '+df['time'].values[i]+' '+df['server_model'].values[i]+' '+df['msg'].values[i]
        string = string.replace('  ', ' ')
        f.write(string)
        f.write('\n')

100%|█████████████████████████████████| 10991/10991 [00:00<00:00, 178142.03it/s]


In [3]:
log_format = '<source> <sn> <Date> <Time> <server_model> <Content>'
parser = LogParser(indir='./dataset/v1/', outdir='./dataset/v1/', log_format=log_format, keep_para=True, tau=1.5)
parser.parse('log.log')

Parsing file: ./dataset/v1/log.log
load log ...


100%|███████████████████████████████| 493527/493527 [00:02<00:00, 188517.09it/s]


parse log ...


get param: 100%|████████████████████| 493527/493527 [00:03<00:00, 145097.68it/s]


Parsing done. [Time taken: 0:00:21.833907]


In [4]:
df = pd.read_csv('./dataset/v1/log.log_templates.csv')
EventId2num = {}
for num, EventId in enumerate(df['EventId'].values):
    EventId2num[EventId] = num

In [5]:
df = pd.read_csv('./dataset/v1/log.log_structured.csv')
df['timestamp'] = df.apply(lambda x: x['Date']+' '+x['Time'], axis=1)
df['EventId'] = df['EventId'].apply(lambda x: EventId2num[x])

server_model2id = {}
for i, sm in enumerate(sorted(list(set(df['server_model'].values)))):
    server_model2id[sm] = i
df['server_model'] = df['server_model'].apply(lambda x: server_model2id[x])


df.head()

,LineId,source,sn,Date,Time,server_model,Content,EventId,EventTemplate,ParameterList,timestamp
0,1,train,SERVER_25698,2020-10-09,08:32:21,0,system state asserted,0,system state asserted,[],2020-10-09 08:32:21
1,2,train,SERVER_25698,2020-10-09,07:43:48,0,system state asserted,0,system state asserted,[],2020-10-09 07:43:48
2,3,train,SERVER_25698,2020-10-09,08:16:22,0,system state asserted,0,system state asserted,[],2020-10-09 08:16:22
3,4,train,SERVER_25698,2020-10-09,05:46:41,0,system state asserted,0,system state asserted,[],2020-10-09 05:46:41
4,5,train,SERVER_25698,2020-10-09,12:59:13,0,system state asserted,0,system state asserted,[],2020-10-09 12:59:13


In [6]:
train = df[df['source']=='train']
train[['sn', 'timestamp', 'server_model', 'EventId', 'ParameterList']].to_csv('./dataset/v1/train.csv', index=False)
test = df[df['source']=='test']
test[['sn', 'timestamp', 'server_model', 'EventId', 'ParameterList']].to_csv('./dataset/v1/test.csv', index=False)